In [1]:
import pandas as pd

gps = pd.read_csv("northern-taiwan.csv", low_memory = False)
data = pd.read_csv("./Locater/queryed.csv", low_memory = False)
gps = gps.rename(columns={'lat': 'y', 'lon': 'x'})
display(gps.head(2),data.head(2))

,station_code,construction_id,station_name_tw,station_name_en,line_code,line_name,address,y,x
0,BR01,BR13,動物園,Taipei Zoo,BR,文湖線,台北市文山區新光路二段32號,24.998197,121.579338
1,BR02,BR12,木柵,Muzha,BR,文湖線,台北市文山區木柵路四段135號,24.998241,121.573145


,p_name,Address,Response_Address,x,y
0,30創富,臺北市萬華區菜園里12鄰康定路84號,臺北市萬華區菜園里12鄰康定路84號,121.502230,25.041989
1,55TIMELESS,臺北市信義區安康里9鄰信義路五段57號,臺北市信義區安康里9鄰信義路五段57號,121.569432,25.033321


In [2]:
gps.tail(10)

,station_code,construction_id,station_name_tw,station_name_en,line_code,line_name,address,y,x
146,A12,A12,機場第一航廈,Airport Terminal 1,A,機場線,桃園市大園區航站南路17之1號B1,25.081385,121.235329
147,A13,A13,機場第二航廈,Airport Terminal 2,A,機場線,桃園市大園區航站南路9之1號B2,25.077270,121.230063
148,A14a,A14a,機場旅館,Airport Hotel,A,機場線,桃園市大園區航站南路1之2號,25.069260,121.218750
149,A15,A15,大園,Dayuan,A,機場線,桃園市大園區橫湳一路50號,25.056047,121.208292
150,A16,A16,橫山,Hengshan,A,機場線,桃園市大園區大竹南路1180號,25.046028,121.204502
151,A17,A17,領航,Linghang,A,機場線,桃園市大園區領航北路四段351號,25.018970,121.213687
152,A18,A18,高鐵桃園站,Taoyuan HSR Station,A,機場線,桃園市中壢區高鐵北路一段5號,25.013762,121.211973
153,A19,A19,桃園體育園區,Taoyuan Sports Park,A,機場線,桃園市中壢區高鐵南路二段350號,25.001996,121.201306
154,A20,A20,興南,Xingnan,A,機場線,桃園市中壢區中豐北路一段685號,24.980249,121.214073
155,A21,A21,環北,Huanbei,A,機場線,桃園市中壢區中豐北路一段26號,24.967381,121.219749


In [3]:
from sklearn.neighbors import BallTree
import numpy as np
import math

def deg2rad(deg):
    return deg * (math.pi / 180)

points  = np.array(data[["x","y"]])
stations = np.array(gps[["x","y"]])

# 將經緯度轉換為弧度
points_rad = np.radians(points)
stations_rad = np.radians(stations)

# 使用BallTree進行最近鄰查找，使用地球半徑（6371公里）
tree = BallTree(stations_rad, metric='haversine')
distances, indices = tree.query(points_rad, k=1)

# 將Haversine距離轉換為公里
distances_in_km = distances * 6371

# 創建DataFrame
df = pd.DataFrame({
    'point_lat': points[:, 0],
    'point_lon': points[:, 1],
    'nearest_station_index': indices.flatten(),
    'distance_to_nearest_station_km': distances_in_km.flatten()
})

# 顯示DataFrame
display(df)

,point_lat,point_lon,nearest_station_index,distance_to_nearest_station_km
0,121.502230,25.041989,107,0.472651
1,121.569432,25.033321,24,0.032763
2,121.523874,25.098205,39,0.320216
3,121.578142,25.077096,16,0.380782
4,121.527835,25.047751,110,0.541932
...,...,...,...,...
762,121.530141,25.062252,80,0.369162
763,121.530141,25.062252,80,0.369162
764,121.514973,25.024090,75,0.599588
765,121.545682,24.999786,57,0.719256


In [11]:
merged_df = pd.merge(df, gps, how='left', left_on='nearest_station_index', right_index=True)
tmp = pd.DataFrame()
tmp["dist_MRT"] =  merged_df['distance_to_nearest_station_km']*1000 
tmp["station"] =  merged_df['station_name_tw'] +merged_df['station_name_en'] 
tmp.head()

,dist_MRT,station
0,472.651236,龍山寺Longshan Temple
1,32.762523,象山Xiangshan
2,320.216395,芝山Zhishan
3,380.781974,港墘Gangqian
4,541.931862,善導寺Shandao Temple


In [12]:
output = pd.merge(data, tmp, how='left', left_index=True, right_index=True)
output.to_csv("distant.csv", index = False)

In [13]:
merged_df

,point_lat,point_lon,nearest_station_index,distance_to_nearest_station_km,station_code,construction_id,station_name_tw,station_name_en,line_code,line_name,address,y,x
0,121.502230,25.041989,107,0.472651,BL10,BL5,龍山寺,Longshan Temple,BL,板南線,台北市萬華區西園路一段153號,25.035280,121.499826
1,121.569432,25.033321,24,0.032763,R02,R5,象山,Xiangshan,R,淡水信義線,台北市信義區信義路五段152號B1,25.032830,121.569576
2,121.523874,25.098205,39,0.320216,R17,R21,芝山,Zhishan,R,淡水信義線,台北市士林區福國路70號,25.103060,121.522514
3,121.578142,25.077096,16,0.380782,BR17,B4,港墘,Gangqian,BR,文湖線,台北市內湖區內湖路一段663號,25.080028,121.575081
4,121.527835,25.047751,110,0.541932,BL13,BL8,善導寺,Shandao Temple,BL,板南線,台北市中正區忠孝東路一段58號B1,25.044823,121.523208
...,...,...,...,...,...,...,...,...,...,...,...,...,...
762,121.530141,25.062252,80,0.369162,O09,O11,行天宮,Xingtian Temple,O,中和新蘆線,台北市中山區松江路316號B1,25.059718,121.533185
763,121.530141,25.062252,80,0.369162,O09,O11,行天宮,Xingtian Temple,O,中和新蘆線,台北市中山區松江路316號B1,25.059718,121.533185
764,121.514973,25.024090,75,0.599588,O04,O16,頂溪,Dingxi,O,中和新蘆線,新北市永和區永和路二段168號B1,25.013821,121.515485
765,121.545682,24.999786,57,0.719256,G05,G5,景美,Jingmei,G,松山新店線,台北市文山區羅斯福路六段393號,24.992128,121.540604


In [14]:
output

,p_name,Address,Response_Address,x,y,dist_MRT,station
0,30創富,臺北市萬華區菜園里12鄰康定路84號,臺北市萬華區菜園里12鄰康定路84號,121.502230,25.041989,472.651236,龍山寺Longshan Temple
1,55TIMELESS,臺北市信義區安康里9鄰信義路五段57號,臺北市信義區安康里9鄰信義路五段57號,121.569432,25.033321,32.762523,象山Xiangshan
2,BR4,臺北士林區舊佳里文林路584號,臺北市士林區舊佳里22鄰文林路582號,121.523874,25.098205,320.216395,芝山Zhishan
3,ForestPark吉美富徠,臺北市內湖區江南街65巷25號之1,臺北市內湖區港墘里5鄰江南街65巷25之1號,121.578142,25.077096,380.781974,港墘Gangqian
4,H24雲端,臺北市中山區正守里12鄰長安東路一段52巷10號,臺北市中山區正守里12鄰長安東路一段52巷10號,121.527835,25.047751,541.931862,善導寺Shandao Temple
...,...,...,...,...,...,...,...
762,鼎吉中山,臺北市中山區吉林路316號,臺北市中山區新生里13鄰吉林路316號,121.530141,25.062252,369.162211,行天宮Xingtian Temple
763,鼎吉中山VISION,臺北市中山區吉林路316號,臺北市中山區新生里13鄰吉林路316號,121.530141,25.062252,369.162211,行天宮Xingtian Temple
764,鼎吉水岸,臺北市中正區螢雪里12鄰水源路165之2號,臺北市中正區螢雪里12鄰水源路165之2號,121.514973,25.024090,599.588334,頂溪Dingxi
765,鼎隆苑,臺北市文山區興豐里10鄰興隆路二段25巷20之1號,臺北市文山區興豐里10鄰興隆路二段25巷20之1號,121.545682,24.999786,719.256044,景美Jingmei
